In [4]:
import re
import datetime
import pandas as pd
import numpy as np
import requests
import io
import json
import random
from main.models import *
from report.models import *
from django.db.models import Sum
from random import randint
from base64 import b64encode, b64decode
from django.contrib.auth.hashers import make_password
from django.core.files.base import ContentFile
from decimal import Decimal
from datetime import datetime, timedelta

try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
    from io import BytesIO

In [5]:
month=5
year=2020
sold_to_list = list(set(DailySessionllyBusinessllySale.objects.all().values_list('sold_to', flat=True)))
union_list = list(set(DailySessionllyBusinessllySale.objects.all().values_list('union', flat=True)))
    
for session in Session.objects.all():
    if MonthlySessionllyRoutellySale.objects.filter(session_id=session.id, month=month, year=year).exists():
        MonthlySessionllyRoutellySale.objects.filter(session_id=session.id, month=month, year=year).delete()
        print("Order deleted for date {} - {}".format(month, session.name))
        for sold_to in sold_to_list:
            for route in Route.objects.filter(session=session):
                route_sales = DailySessionllyRoutellySale.objects.filter(delivery_date__month=month, delivery_date__year=year, route=route, sold_to=sold_to)
                 # Create an User table entry for this Agent
                if route_sales.count() > 0:
                    msrs, created =  MonthlySessionllyRoutellySale.update_or_create(
                        month=month,
                        year=year,
                        session=session,
                        route=route,
                        sold_to=sold_to,
                        defaults = {
                            'created_by':User.objects.get(username='kutobot'),
                            'modified_by':User.objects.get(username='kutobot'),
                            'total_cost' :route_sales.aggregate(Sum('total_cost'))['total_cost__sum'],
                        }

                    )
                    if created:
                        print("\tRow Created: {}\t{}\t{}".format(msrs.delivery_date, msrs.session, msrs.route.name))
                    else:
                        print("\tRow Exists: {}\t{}\t{}".format(msrs.delivery_date, msrs.session, msrs.route.name))


                    # Now fill the route wise sum

                    # Milk
                    if getattr(MonthlySessionllyRoutellySale, 'tm500_pkt', True):
                        msrs.tm500_pkt = route_sales.aggregate(Sum('tm500_pkt'))['tm500_pkt__sum']
                        msrs.tm500_litre = route_sales.aggregate(Sum('tm500_litre'))['tm500_litre__sum']
                        msrs.tm500_cost = route_sales.aggregate(Sum('tm500_cost'))['tm500_cost__sum']
                        msrs.tm500_unit_price = route_sales.aggregate(Sum('tm500_unit_price'))['tm500_unit_price__sum']

                    if getattr(MonthlySessionllyRoutellySale, 'std250_pkt', True):
                        msrs.std250_pkt = route_sales.aggregate(Sum('std250_pkt'))['std250_pkt__sum']
                        msrs.std250_litre = route_sales.aggregate(Sum('std250_litre'))['std250_litre__sum']
                        msrs.std250_cost = route_sales.aggregate(Sum('std250_cost'))['std250_cost__sum']
                        msrs.std250_unit_price = route_sales.aggregate(Sum('std250_unit_price'))['std250_unit_price__sum']


                    if getattr(MonthlySessionllyRoutellySale, 'std500_pkt', True):
                        msrs.std500_pkt = route_sales.aggregate(Sum('std500_pkt'))['std500_pkt__sum']
                        msrs.std500_litre = route_sales.aggregate(Sum('std500_litre'))['std500_litre__sum']
                        msrs.std500_cost = route_sales.aggregate(Sum('std500_cost'))['std500_cost__sum']
                        msrs.std500_unit_price = route_sales.aggregate(Sum('std500_unit_price'))['std500_unit_price__sum']

                    if getattr(MonthlySessionllyRoutellySale, 'fcm500_pkt', True):
                        msrs.fcm500_pkt = route_sales.aggregate(Sum('fcm500_pkt'))['fcm500_pkt__sum']   
                        msrs.fcm500_litre = route_sales.aggregate(Sum('fcm500_litre'))['fcm500_litre__sum']   
                        msrs.fcm500_cost = route_sales.aggregate(Sum('fcm500_cost'))['fcm500_cost__sum']   
                        msrs.fcm500_unit_price = route_sales.aggregate(Sum('fcm500_unit_price'))['fcm500_unit_price__sum']   

                    if getattr(MonthlySessionllyRoutellySale, 'fcm1000_pkt', True):
                        msrs.fcm1000_pkt = route_sales.aggregate(Sum('fcm1000_pkt'))['fcm1000_pkt__sum']   
                        msrs.fcm1000_litre = route_sales.aggregate(Sum('fcm1000_litre'))['fcm1000_litre__sum']   
                        msrs.fcm1000_cost = route_sales.aggregate(Sum('fcm1000_cost'))['fcm1000_cost__sum']   
                        msrs.fcm1000_unit_price = route_sales.aggregate(Sum('fcm1000_unit_price'))['fcm1000_unit_price__sum']


                    if getattr(MonthlySessionllyRoutellySale, 'tmcan', True):
                        msrs.tmcan = route_sales.aggregate(Sum('tmcan'))['tmcan__sum']
                        msrs.tmcan_litre = route_sales.aggregate(Sum('tmcan_litre'))['tmcan_litre__sum']
                        msrs.tmcan_cost = route_sales.aggregate(Sum('tmcan_cost'))['tmcan_cost__sum']
                        msrs.tmcan_unit_price = route_sales.aggregate(Sum('tmcan_unit_price'))['tmcan_unit_price__sum']


                    if getattr(MonthlySessionllyRoutellySale, 'smcan', True):
                        msrs.smcan = route_sales.aggregate(Sum('smcan'))['smcan__sum']
                        msrs.smcan_litre = route_sales.aggregate(Sum('smcan_litre'))['smcan_litre__sum']
                        msrs.smcan_cost = route_sales.aggregate(Sum('smcan_cost'))['smcan_cost__sum']
                        msrs.smcan_unit_price = route_sales.aggregate(Sum('smcan_unit_price'))['smcan_unit_price__sum']

                    if getattr(MonthlySessionllyRoutellySale, 'fcmcan', True):
                        msrs.fcmcan = route_sales.aggregate(Sum('fcmcan'))['fcmcan__sum']
                        msrs.fcmcan_litre = route_sales.aggregate(Sum('fcmcan_litre'))['fcmcan_litre__sum']
                        msrs.fcmcan_cost = route_sales.aggregate(Sum('fcmcan_cost'))['fcmcan_cost__sum']
                        msrs.fcmcan_unit_price = route_sales.aggregate(Sum('fcmcan_unit_price'))['fcmcan_unit_price__sum']


                    # Curd    
                    if getattr(MonthlySessionllyRoutellySale, 'curd500_pkt', True):
                        msrs.curd500_pkt = route_sales.aggregate(Sum('curd500_pkt'))['curd500_pkt__sum']
                        msrs.curd500_kgs = route_sales.aggregate(Sum('curd500_kgs'))['curd500_kgs__sum']
                        msrs.curd500_cost = route_sales.aggregate(Sum('curd500_cost'))['curd500_cost__sum']
                        msrs.curd500_unit_price = route_sales.aggregate(Sum('curd500_unit_price'))['curd500_unit_price__sum']


                    if getattr(MonthlySessionllyRoutellySale, 'curd150_pkt', True):
                        msrs.curd150_pkt = route_sales.aggregate(Sum('curd150_pkt'))['curd150_pkt__sum']
                        msrs.curd150_kgs = route_sales.aggregate(Sum('curd150_kgs'))['curd150_kgs__sum']
                        msrs.curd150_cost = route_sales.aggregate(Sum('curd150_cost'))['curd150_cost__sum']
                        msrs.curd150_unit_price = route_sales.aggregate(Sum('curd150_unit_price'))['curd150_unit_price__sum']

                    if getattr(MonthlySessionllyRoutellySale, 'cupcurd_box', True):
                        msrs.cupcurd_box = route_sales.aggregate(Sum('cupcurd_box'))['cupcurd_box__sum']
                        msrs.cupcurd_count = route_sales.aggregate(Sum('cupcurd_count'))['cupcurd_count__sum']
                        msrs.cupcurd_kgs = route_sales.aggregate(Sum('cupcurd_kgs'))['cupcurd_kgs__sum']
                        msrs.cupcurd_cost = route_sales.aggregate(Sum('cupcurd_cost'))['cupcurd_cost__sum']
                        msrs.cupcurd_unit_price = route_sales.aggregate(Sum('cupcurd_unit_price'))['cupcurd_unit_price__sum']

                    if getattr(MonthlySessionllyRoutellySale, 'curd_bucket', True):
                        msrs.curd_bucket = route_sales.aggregate(Sum('curd_bucket'))['curd_bucket__sum']
                        msrs.curd_bucket_kgs = route_sales.aggregate(Sum('curd_bucket_kgs'))['curd_bucket_kgs__sum']
                        msrs.curd_bucket_cost = route_sales.aggregate(Sum('curd_bucket_cost'))['curd_bucket_cost__sum']
                        msrs.curd_bucket_unit_price = route_sales.aggregate(Sum('curd_bucket_unit_price'))['curd_bucket_unit_price__sum']

                    if getattr(MonthlySessionllyRoutellySale, 'lassi200_pkt', True):
                        msrs.lassi200_pkt = route_sales.aggregate(Sum('lassi200_pkt'))['lassi200_pkt__sum']
                        msrs.lassi200_kgs = route_sales.aggregate(Sum('lassi200_kgs'))['lassi200_kgs__sum']
                        msrs.lassi200_cost = route_sales.aggregate(Sum('lassi200_cost'))['lassi200_cost__sum']
                        msrs.lassi200_unit_price = route_sales.aggregate(Sum('lassi200_unit_price'))['lassi200_unit_price__sum']

                    if getattr(MonthlySessionllyRoutellySale, 'buttermilk200_pkt', True):
                        msrs.buttermilk200_pkt = route_sales.aggregate(Sum('buttermilk200_pkt'))['buttermilk200_pkt__sum']
                        msrs.buttermilk200_litre = route_sales.aggregate(Sum('buttermilk200_litre'))['buttermilk200_litre__sum']
                        msrs.buttermilk200_cost = route_sales.aggregate(Sum('buttermilk200_cost'))['buttermilk200_cost__sum']
                        msrs.buttermilk200_unit_price = route_sales.aggregate(Sum('buttermilk200_unit_price'))['buttermilk200_unit_price__sum']
                    
                    
                    msrs.tm_litre = route_sales.aggregate(Sum('tm_litre'))['tm_litre__sum']
                    msrs.tm_cost = route_sales.aggregate(Sum('tm_cost'))['tm_cost__sum']
                    msrs.sm_litre = route_sales.aggregate(Sum('sm_litre'))['sm_litre__sum']
                    msrs.sm_cost = route_sales.aggregate(Sum('sm_cost'))['sm_cost__sum']
                    msrs.fcm_litre = route_sales.aggregate(Sum('fcm_litre'))['fcm_litre__sum']
                    msrs.fcm_cost = route_sales.aggregate(Sum('fcm_cost'))['fcm_cost__sum']
                    msrs.milk_litre = route_sales.aggregate(Sum('milk_litre'))['milk_litre__sum']
                    msrs.milk_cost = route_sales.aggregate(Sum('milk_cost'))['milk_cost__sum']
                    msrs.curd_kgs = route_sales.aggregate(Sum('curd_kgs'))['curd_kgs__sum']
                    msrs.curd_cost = route_sales.aggregate(Sum('curd_cost'))['curd_cost__sum']
                    msrs.buttermilk_cost = route_sales.aggregate(Sum('buttermilk_cost'))['buttermilk_cost__sum']
                    msrs.buttermilk_litre = route_sales.aggregate(Sum('buttermilk_litre'))['buttermilk_litre__sum']
                    msrs.lassi_litre = route_sales.aggregate(Sum('lassi_litre'))['lassi_litre__sum']
                    msrs.lassi_cost = route_sales.aggregate(Sum('lassi_cost'))['lassi_cost__sum']
                    msrs.fermented_products_litre = route_sales.aggregate(Sum('fermented_products_litre'))['fermented_products_litre__sum']
                    msrs.fermented_products_cost = route_sales.aggregate(Sum('fermented_products_cost'))['fermented_products_cost__sum']
                    msrs.total_litre = route_sales.aggregate(Sum('total_litre'))['total_litre__sum']

                    msrs.save()


            print(sold_to)

In [3]:
SaleGroup.objects.filter(date='2020-05-01', business__code=256)[0].route.name

'A - PODANU_MOR'

In [7]:
DailySessionllyBusinessllySale.objects.filter(delivery_date='2020-05-01', business__code=256, sold_to='ICustomer')[0].route.name

'W - ROUTE_MOR'